# Raw Transformed Processing

## Preparation

In [1]:
import pandas as pd
import json
import os
from random import choice

pd.set_option('display.max_columns', None)

## Data Information & Description

In [2]:
directories = []
for f in os.scandir('../data/raw/'):
    if 'transfers_by_day_clean_' in f.name:
        directories.append(f.name)

transfers_by_day_clean_latest = sorted(directories)[-1]
print(transfers_by_day_clean_latest)

transfers_by_day_clean_20230326135132


In [74]:
# sample from a random year
year = choice([i for i in range(2000, 2015)])
df = pd.read_parquet(f'../data/raw/{transfers_by_day_clean_latest}/y={year}/', engine='pyarrow')
df['y'] = str(year)
df.sample(25)

,portrait_url,name,player_id,position,age,nationalities,left_club_url,left_club_name,left_club_name_alt,left_club_league_country_url,left_club_league_country_name,left_club_league_name,left_club_league_name_alt,join_club_url,join_club_name,join_club_name_alt,join_club_league_country_url,join_club_league_country_name,join_club_league_name,join_club_league_name_alt,market_value,fee,loan_fee,transfer_url,transfer_date,ingested_at,m,d,y
24498,https://img.a.transfermarkt.technology/portrai...,Tommaso Colombaretti,102771,Centre-Back,34,"[{""name"": ""Italy"", ""url"": ""https://tmssl.akama...",https://tmssl.akamaized.net/images/wappen/tiny...,Foggia,Calcio Foggia 1920,https://tmssl.akamaized.net/images/flagge/tiny...,Italy,Serie C - C,Serie C - C,https://tmssl.akamaized.net/images/wappen/tiny...,Without Club,Without Club,,,,,€75k,-,,/jumplist/transfers/spieler/102771/transfer_id...,2014-07-01,2023-03-25 10:00:05,7,1,2014
27497,https://img.a.transfermarkt.technology/portrai...,Maksim Fedoseev,288154,Defensive Midfield,22,"[{""name"": ""Russia"", ""url"": ""https://tmssl.akam...",https://tmssl.akamaized.net/images/wappen/tiny...,Rubin 2,Rubin 2 Kazan,https://tmssl.akamaized.net/images/flagge/tiny...,Russia,2 League Group 4,2 League Group 4,https://tmssl.akamaized.net/images/wappen/tiny...,Neftekhimik,Neftekhimik Nizhnekamsk,https://tmssl.akamaized.net/images/flagge/very...,Russia,2 League Group 4,2 League Group 4,€25k,free transfer,,/jumplist/transfers/spieler/288154/transfer_id...,2014-07-01,2023-03-25 08:01:05,7,1,2014
28717,https://img.a.transfermarkt.technology/portrai...,Paul Gallagher,10501,Central Midfield,29,"[{""name"": ""Scotland"", ""url"": ""https://tmssl.ak...",https://tmssl.akamaized.net/images/wappen/tiny...,Leicester,Leicester City,https://tmssl.akamaized.net/images/flagge/tiny...,England,Premier League,Premier League,https://tmssl.akamaized.net/images/wappen/tiny...,Preston,Preston North End,https://tmssl.akamaized.net/images/flagge/very...,England,League One,League One,€400k,loan transfer,,/jumplist/transfers/spieler/10501/transfer_id/...,2014-07-04,2023-03-24 12:11:47,7,4,2014
35523,https://img.a.transfermarkt.technology/portrai...,Julio Bado,299107,Defensive Midfield,31,"[{""name"": ""Gibraltar"", ""url"": ""https://tmssl.a...",https://tmssl.akamaized.net/images/wappen/tiny...,Lynx FC,Lynx FC,https://tmssl.akamaized.net/images/flagge/tiny...,Gibraltar,Gibraltar Football League,Gibraltar Football League,https://tmssl.akamaized.net/images/wappen/tiny...,Manchester 62,Manchester 62 FC,https://tmssl.akamaized.net/images/flagge/very...,Gibraltar,Gibraltar Football League,Gibraltar Football League,€25k,free transfer,,/jumplist/transfers/spieler/299107/transfer_id...,2014-08-05,2023-03-24 12:56:30,8,5,2014
41638,https://img.a.transfermarkt.technology/portrai...,Ermal Sako,240179,Midfield,25,"[{""name"": ""Albania"", ""url"": ""https://tmssl.aka...",https://tmssl.akamaized.net/images/wappen/tiny...,KF Albpetrol,KF Albpetrol Patosi,https://tmssl.akamaized.net/images/flagge/tiny...,Albania,,,https://tmssl.akamaized.net/images/wappen/tiny...,FK Bylis,FK Bylis,https://tmssl.akamaized.net/images/flagge/very...,Albania,,,€25k,free transfer,,/jumplist/transfers/spieler/240179/transfer_id...,2014-09-26,2023-03-24 14:07:33,9,26,2014
27549,https://img.a.transfermarkt.technology/portrai...,Andrei Ceaca,291442,Centre-Forward,24,"[{""name"": ""Romania"", ""url"": ""https://tmssl.aka...",https://tmssl.akamaized.net/images/wappen/tiny...,Sanatatea,Sanatatea Cluj,https://tmssl.akamaized.net/images/flagge/tiny...,Romania,,,https://tmssl.akamaized.net/images/wappen/tiny...,Sticla A. Turda,Sticla Ariesul Turda,https://tmssl.akamaized.net/images/flagge/very...,Romania,,,€25k,free transfer,,/jumplist/transfers/spieler/291442/transfer_id...,2014-07-01,2023-03-25 08:01:16,7,1,2014
14694,https://img.a.transfermarkt.technology/portrai...,Evgeniy Lebedev,285647,Centre-Forward,19,"[{""name"": ""Belarus"", ""url"": ""https://tmssl.aka...",https://tms

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41804 entries, 0 to 40501
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   portrait_url                   41804 non-null  object  
 1   name                           41804 non-null  object  
 2   player_id                      41804 non-null  object  
 3   position                       41804 non-null  object  
 4   age                            41804 non-null  object  
 5   nationalities                  41804 non-null  object  
 6   left_club_url                  41804 non-null  object  
 7   left_club_name                 41804 non-null  object  
 8   left_club_name_alt             41804 non-null  object  
 9   left_club_league_country_url   41804 non-null  object  
 10  left_club_league_country_name  41804 non-null  object  
 11  left_club_league_name          41804 non-null  object  
 12  left_club_league_name_alt      4

## Create Transformed Dataset

In [68]:
# unnecessary features to drops
drop_features = [
    'portrait_url',
    'left_club_url',
    'left_club_league_country_url',
    'join_club_url',
    'join_club_league_country_url',
    'ingested_at',
    'transfer_url',
    'nationalities',
]

def nationalities_parse(json_string, index=0):
    
    try:
        json_string = json_string.replace('d"Ivoire', 'd Ivoire')
        
        json_list_dict = json.loads(json_string)
        if len(json_list_dict) > index:
            return json_list_dict[index]['name']
        else:
            return 'EMPTY'
    except Exception as e:
        print(json_string)
        print(e)

def value_parse(value_string):
    
    if (value_string):
        value = value_string.replace('€', '')
        if ("Th." in value):
            value = value.replace("Th.", '')
            value = float(value) * 1000
        elif ("k" in value):
            value = value.replace("k", '')
            value = float(value) * 1000
        elif ("m" in value):
            value = value.replace("m", '')
            value = float(value) * 1000000
        else:
            return 0

        return int(value)
    else:
        return 0   


In [76]:
from datetime import datetime
ymdhms = datetime.now().strftime('%Y%m%d%H%M%S')

for year in range(2000, 2015):
    df = pd.read_parquet(f'../data/raw/{transfers_by_day_clean_latest}/y={year}/', engine='pyarrow')
    df['y'] = str(year)

    df['is_free_transfer'] = df['fee'].apply(lambda fee: 1 if (str(fee).strip() == 'free transfer') else 0)
    df['is_loan_transfer'] = df['fee'].apply(lambda fee: 1 if (str(fee).strip() == 'loan transfer') else 0)
    df['is_draft'] = df['fee'].apply(lambda fee: 1 if (str(fee).strip() == 'draft') else 0)

    df['market_value'] = df['market_value'].apply(lambda value: value_parse(value))
    df['fee'] = df['fee'].apply(lambda value: value_parse(value))
    df['loan_fee'] = df['loan_fee'].apply(lambda value: value_parse(value))
    df['nationality_1'] = df['nationalities'].apply(lambda nats: nationalities_parse(nats, 0))
    df['nationality_2'] = df['nationalities'].apply(lambda nats: nationalities_parse(nats, 1))

    df['left_club_league_country_name'] = df['left_club_league_country_name'].apply(lambda value: value if value else 'EMPTY')
    df['left_club_league_name'] = df['left_club_league_name'].apply(lambda value: value if value else 'EMPTY')
    df['left_club_league_name_alt'] = df['left_club_league_name_alt'].apply(lambda value: value if value else 'EMPTY')
    df['join_club_league_country_name'] = df['join_club_league_country_name'].apply(lambda value: value if value else 'EMPTY')
    df['join_club_league_name'] = df['join_club_league_name'].apply(lambda value: value if value else 'EMPTY')
    df['join_club_league_name_alt'] = df['join_club_league_name_alt'].apply(lambda value: value if value else 'EMPTY')

    df = df.drop(drop_features, axis=1)
    
    df.to_parquet(f'../data/raw/transfers_by_day_transformed_{ymdhms}/', partition_cols=['y', 'm', 'd'])
    